In [ ]:
#Import the libraries
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import yfinance as yf

In [ ]:
start_date2 = "2021-07-16"
end_date2 = "2021-07-28"
# Fetch historic data from yfiance
#df_historic2015_2019 = yfinance_tickers_data("NG=F", start_date2, end_date2, drop_extra_cols = True )
df = yf.download("ba", start="2017-01-01", end="2021-07-29")
df.tail()

In [ ]:
#get the stock quote
#df = web.DataReader('BA', data_source='yahoo', start='2021-07-16', end='2021-07-28') 

In [ ]:
#show data
df.tail(10)

In [ ]:
# get the number of rows and colums inthe data set
df.shape

In [ ]:
import plotly.express as px

df = df.reset_index()
fig = px.line(df, x='Date', y="Close")
fig.show()

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
import yfinance as yf

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# assume you have a "long-form" data frame

df = yf.download("ba", start="2017-01-01", end="2021-07-29").reset_index()
fig = px.line(df, x='Date', y="Close")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=False, port=8070)

[*********************100%***********************]  1 of 1 completed
Dash is running on http://127.0.0.1:8070/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8070/ (Press CTRL+C to quit)
127.0.0.1 - - [28/Jul/2021 20:52:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 20:52:56] "GET /_dash-component-suites/dash/deps/polyfill@7.v1_21_0m1627149480.12.1.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 20:52:56] "GET /_dash-component-suites/dash/deps/prop-types@15.v1_21_0m1627149480.7.2.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 20:52:56] "GET /_dash-component-suites/dash/deps/react-dom@16.v1_21_0m1627149480.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 20:52:56] "GET /_dash-component-suites/dash_html_components/dash_html_components.v1_1_4m1627149479.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 20:52:56] "GET /_dash-component-suites/dash/deps/react@16.v1_21_0m1627149480.14.0.min.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 20:52:56] "GET /_dash-component-suites/dash_core_components/dash_core_components-shared.v1_17_1m1627149480.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Jul/2021 20:52:56] "GE

In [ ]:
# visualize the closing price history
plt.figure(figsize=(16,8))
plt.title('Close Price History')
plt.plot(df['Close'])
plt.xlabel('Date',fontsize=18)
plt.ylabel('Close Price USD ($)',fontsize=18)
plt.show()

In [ ]:
# Create a new datafram with only the "close column"
data = df.filter(['Close'])
#Convert the dataframe to a numpy array
dataset = data.values
#Get the nubmer of rows to train the model on
training_data_len = math.ceil( len(dataset) *.8) # math.ceil is to round up the numbers, training data is 80%
training_data_len

In [ ]:
# Using MinMaxScaler from sklearn to scale the data betweek 0 and 1
scaler = MinMaxScaler(feature_range = (0,1))
scaled_data = scaler.fit_transform(dataset)


In [ ]:
scaled_data

In [ ]:
# create the training datae set
#create the scaled training date set
train_data = scaled_data[0:training_data_len, :]


In [ ]:
#Split the data into x_train and y_train data sets
x_train=[] #training feature
y_train = [] #targt varible
for i in range(60,len(train_data)):
    x_train.append(train_data[i-60:i,0]) # not including i
    y_train.append(train_data[i,0])# 61st value
    if i<= 60:
        print(x_train)
        print(y_train)
        print()

In [ ]:
#Convert x_train and y_train to numpy arrays
x_train, y_train = np.array(x_train), np.array(y_train)

In [ ]:
#Reshape the data into the shape accepted by the LSTM
x_train = np.reshape(x_train, (x_train.shape[0],x_train.shape[1],1))
x_train.shape

In [ ]:
#Build the LSTM network model
model = Sequential()

# Layer 1 
model.add(LSTM(units=50, return_sequences=True,input_shape=(x_train.shape[1],1))) #50 Neurons

# Layer 2
model.add(LSTM(units=50, return_sequences=False))

#Output layers
model.add(Dense(units=25))
model.add(Dense(units=1))

In [ ]:
#Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Summarize the model
model.summary()

In [ ]:
# Training the model
model.fit(x_train, y_train, batch_size = 1, epochs = 1)

In [ ]:
# Create the testing data set
# Crete a new array containing scaled valueds from index 1543 to 2003
test_data = scaled_data[training_data_len -60:,:]

In [ ]:
# Create the data sets x_test and y_test

In [ ]:
x_test = []
y_test =  dataset[training_data_len : , : ] #Get all of the rows from index 1603 to the rest and all of the columns (in this case it's only column 'Close'), so 2003 - 1603 = 400 rows of data
for i in range(60,len(test_data)):
    x_test.append(test_data[i-60:i,0])

In [ ]:
# convert the data to a numpy array
x_test = np.array(x_test)

In [ ]:
#Reshape the data into the shape accepted by the LSTM
x_test = np.reshape(x_test, (x_test.shape[0],x_test.shape[1],1))

In [ ]:
#Getting the models predicted price values
predictions = model.predict(x_test) 
predictions = scaler.inverse_transform(predictions)#Undo scaling
real = scaler.inverse_transform(y_test.reshape(-1,1))


In [ ]:
# Get the root mean squrared erro(RMSE)--how accurate the model to predit the response, the standard deviatation of the residules. 
rmse=np.sqrt(np.mean(((predictions- y_test)**2)))
rmse

In [ ]:
#Plot/Create the data for the graph
train = data[:training_data_len]
valid = data[training_data_len:]
valid['Predictions'] = predictions


In [ ]:
#Visualize the data
plt.figure(figsize=(16,8))
plt.title('Model')
plt.xlabel('Date', fontsize=18)
plt.ylabel('Close Price USD ($)', fontsize=18)
plt.plot(train['Close'])
plt.plot(valid[['Close', 'Predictions']])
plt.legend(['Train', 'val', 'Predictions'], loc='lower right')
plt.show()

In [ ]:
# show the valid (actural  and predicted price)
valid

In [ ]:
valid.plot()

In [ ]:
# get the quote 

In [ ]:
#Get the quote
apple_quote = web.DataReader('AAPL', data_source='yahoo', start='2012-01-01', end='2021-07-29')
#Create a new dataframe
new_df = apple_quote.filter(['Close'])
#Get teh last 60 day closing price 
last_60_days = new_df[-60:].values
#Scale the data to be values between 0 and 1
last_60_days_scaled = scaler.transform(last_60_days)
#Create an empty list
X_test = []
#Append teh past 60 days
X_test.append(last_60_days_scaled)
#Convert the X_test data set to a numpy array
X_test = np.array(X_test)
#Reshape the data
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
#Get the predicted scaled price
pred_price = model.predict(X_test)
#undo the scaling 
pred_price = scaler.inverse_transform(pred_price)
print(pred_price)

In [ ]:
#Get the quote
apple_quote2 = web.DataReader('AAPL', data_source='yahoo', start='2019-12-18', end='2021-07-29')
print(apple_quote2['Close'])